In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics as skm
import math

In [ ]:
my_data = np.genfromtxt('MultipleLR.csv', delimiter=',')
X =  my_data[:,0:-1]
Y =  my_data[:,[-1]]

In [ ]:
def predict_h(X_h , Theta_h):
    """
    this function takes the 
    features matrix (mxn) matrix
    and the 
    theata vector (nx1) vactor
    ----
    return the predicted values vector (mx1)
    """
    if (X_h.shape[1] == len(Theta_h)):
        return np.dot(X_h, Theta_h)
    else:
        X_h = np.hstack((np.ones((X_h.shape[0],1)),X_h))
        return np.dot(X_h, Theta_h)

In [ ]:
def get_gradient(X_f,Y_f ,Theta_g ):
    """
    this function takes the 
    features matrix (mxn) matrix
    and the 
    theata vector (nx1) vactor
    ------
    it uses the function predict_h()
    -------
    return
    the gradient vector (nx1) vactor
    
    """
    m = X_f.shape[0]
    #np.dot(X_h, Theta_h)
    y_hat = predict_h(X_f, Theta_g)
    loss = y_hat - Y_f
    return (1/m)*np.dot(X_f.T,(loss))

In [ ]:
def stop_criteria_gradient(grad,limits = .0001):
    return abs(np.sum(grad)) <= limits

In [ ]:
def preparethedata(features,target,batch_size):
    """
    return 
    features
    batch_size 
    No_of_batches 
    full_features 
    full_target
    """
    features = np.hstack((np.ones((features.shape[0],1)),features))
       
    if batch_size == None:
        batch_size = features.shape[0]
        
     #split into batches
    No_of_batches = math.ceil(features.shape[0]/batch_size)
    full_features = np.array_split(features, No_of_batches)
    full_target  = np.array_split(target, No_of_batches)
    return features,batch_size , No_of_batches , full_features , full_target
    


    Adagrad, 
    RMSProp, and 
    Adam algorithm.

## seperate into methods 

In [ ]:
def gradient_descent(features , target
                     , number_of_iterations=1000
                     , learning_rate = 0.0001
                     , batch_size = None
                     , gradlimit = .0001):
    
    thetas = []
    costs = []
    accrs = []
    grads = []
    
    features,batch_size , No_of_batches , full_features , full_target = preparethedata(features,target,batch_size)
    
    #intialize the theta
    theta = np.zeros(features.shape[1]).reshape((features.shape[1],1))
    
    # intialize Y hat for spliting 
    y_hat = full_target.copy()
    
    #stop to max iter
    iter_time = 0
    #loop over the iterations
    for t in range(number_of_iterations):
        
        #loop over the batches
        for i in range(No_of_batches):
            
            y_hat[i] = predict_h(full_features[i],theta)
            g = get_gradient(full_features[i],full_target[i] ,theta)
            theta = theta - learning_rate*g
            thetas.append(theta)
    
            # for full batch or small batches cases
            if (batch_size >=2):
                #accrs
                y_hat[i] = predict_h(full_features[i],theta)
                accrs.append(round(skm.r2_score(full_target[i],y_hat[i]),6))
                
                #calc the cost
                m = full_target[i].shape[0]
                loss = y_hat[i] - full_target[i]
                cost = 1/(2*m)*np.sum(loss**2)
                costs.append(cost)
                #######################
                
        # for 1 by 1 value (Stochastic)
        if (batch_size <2):
            #accrs
            y_hat = np.dot(features, theta)
            accrs.append(round(skm.r2_score(target,y_hat),6))
            ########################
            
            #calc the cost
            m = target.shape[0]
            y_hat = predict_h(features,theta)
            loss = y_hat - target
            cost = 1/(2*m)*np.sum(loss**2)
            costs.append(cost)
            #######################
        
        iter_time += 1
        
        #get final grad after each iteration to add to list 
        g = get_gradient(full_features[i],full_target[i] ,theta)
        grads.append(g)
        
        if stop_criteria_gradient(g,gradlimit):
            break
#     end the for iteration loop
    
    #for extraction purpose 
    thetas = np.array(thetas).reshape(iter_time,features.shape[1])
    
    #end function 
    return theta,thetas,costs, accrs,iter_time,grads

In [ ]:
theta,thetas,costs, accrs,iter_time,grads= gradient_descent(X,Y, number_of_iterations=500
                     , learning_rate = 0.0001)
accrs[-1]

In [ ]:
def gradient_descent_with_momentom(features , target
                     , number_of_iterations = 1000
                     , learning_rate = 0.0001
                     , batch_size = None
                     , momentum_gama = 0
                     , nag = False
                     , gradlimit = .0001):
    thetas = []
    costs = []
    accrs = []
    grads = []
    
    features,batch_size , No_of_batches , full_features , full_target = preparethedata(features,target,batch_size)
    
    #intialize the theta
    theta = np.zeros(features.shape[1]).reshape((features.shape[1],1))
    
    # intialize Y hat for spliting 
    y_hat = full_target.copy()
    
    #stop to max iter
    iter_time = 0
    v_moment = 0
    #loop over the iterations
    for t in range(number_of_iterations):
        
        #loop over the batches
        for i in range(No_of_batches):
            #for moment and nag
            if nag == True:
                theta -= (momentum_gama* v_moment)
                y_hat[i] = predict_h(full_features[i],theta)
                g = get_gradient(full_features[i],full_target[i] ,theta)
                theta = theta - (learning_rate *g)

            else:
                y_hat[i] = predict_h(full_features[i],theta)

            #out of if else nag
            g = get_gradient(full_features[i],full_target[i] ,theta)
            v_moment = momentum_gama*v_moment+(learning_rate*g)
            theta = theta - v_moment
            thetas.append(theta)
    
            # for full batch or small batches cases
            if (batch_size >=2):
                #accrs
                y_hat[i] = predict_h(full_features[i],theta)
                accrs.append(round(skm.r2_score(full_target[i],y_hat[i]),6))
                
                #calc the cost
                m = full_target[i].shape[0]
                loss = y_hat[i] - full_target[i]
                cost = 1/(2*m)*np.sum(loss**2)
                costs.append(cost)
                #######################
                
        # for 1 by 1 value (Stochastic)
        if (batch_size <2):
            #accrs
            y_hat = np.dot(features, theta)
            accrs.append(round(skm.r2_score(target,y_hat),6))
            ########################
            
            #calc the cost
            m = target.shape[0]
            y_hat = predict_h(features,theta)
            loss = y_hat - target
            cost = 1/(2*m)*np.sum(loss**2)
            costs.append(cost)
            #######################
        
        iter_time += 1
        
        #get final grad after each iteration to add to list 
        g = get_gradient(full_features[i],full_target[i] ,theta)
        grads.append(g)
        
        if stop_criteria_gradient(g,gradlimit):
            break
#     end the for iteration loop
    
    #for extraction purpose 
    thetas = np.array(thetas).reshape(iter_time,features.shape[1])
    
    #end function 
    return theta,thetas,costs, accrs,iter_time,grads

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_momentom(X , Y
                     , number_of_iterations = 1000
                     , learning_rate = 0.00001
                     , batch_size = None
                     , momentum_gama = .9
                     , nag = True
                     , gradlimit = .0001)
accrs[-1]

## Adagrad method

In [ ]:
def gradient_descent_with_Adagrad(features , target
                     , number_of_iterations=1000
                     , learning_rate = 0.0001
                     , batch_size = None
                     , gradlimit = .0001):
    thetas = []
    costs = []
    accrs = []
    grads = []
    
    features,batch_size,No_of_batches,full_features,full_target=preparethedata(features,target,batch_size)
    
    #intialize the theta
    theta = np.zeros(features.shape[1]).reshape((features.shape[1],1))
    
    # intialize Y hat for spliting 
    y_hat = full_target.copy()

    v_Adagrad = 0
    #stop to max iter
    iter_time = 0
    #loop over the iterations
    for t in range(number_of_iterations):
        
        #loop over the batches
        for i in range(No_of_batches):
                
            y_hat[i] = predict_h(full_features[i],theta)
            g = get_gradient(full_features[i],full_target[i] ,theta)
            v_Adagrad = v_Adagrad + g**2
            theta = theta - learning_rate*g*(1/(np.sqrt(v_Adagrad)+1e-5))
            thetas.append(theta)
    
            # for full batch or small batches cases
            if (batch_size >=2):
                y_hat[i] = predict_h(full_features[i],theta)
                accrs.append(round(skm.r2_score(full_target[i],y_hat[i]),6))
                
        
        # for 1 by 1 value (Stochastic)
        if (batch_size <2):
            y_hat = np.dot(features, theta)
            accrs.append(round(skm.r2_score(target,y_hat),6))
            #calc the cost
            m = target.shape[0]
            y_hat = predict_h(features,theta)
            loss = y_hat - target
            cost = 1/(2*m)*np.sum(loss**2)
            costs.append(cost)
        iter_time += 1
        
        #get final grad after each iteration to add to list 
        g = get_gradient(full_features[i],full_target[i] ,theta)
        grads.append(g)
        
        if stop_criteria_gradient(g,gradlimit):
            break
#     end the for iteration loop
    
    #for extraction purpose 
    thetas = np.array(thetas).reshape(iter_time,features.shape[1])
    
    #end function 
    return theta,thetas,costs, accrs,iter_time,grads

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_Adagrad(X , Y
                     , number_of_iterations=1000
                     , learning_rate = .1
                     , batch_size = None
                     , gradlimit = .00001)
accrs[-1]

### gradient descent with rmsprop

In [ ]:
def gradient_descent_with_rmsprop(features , target
                      , number_of_iterations=1000
                      , learning_rate = 0.0001
                      , batch_size = None
                      , beta_rmsprop = .98
                      , gradlimit = .0001):
    thetas = []
    costs = []
    accrs = []
    grads = []
    
    features,batch_size,No_of_batches,full_features,full_target=preparethedata(features,target,batch_size)
    
    #intialize the theta
    theta = np.zeros(features.shape[1]).reshape((features.shape[1],1))
    
    # intialize Y hat for spliting 
    y_hat = full_target.copy()

    v_rmsprop = 0
    #stop to max iter
    iter_time = 0
    #loop over the iterations
    for t in range(number_of_iterations):
        
        #loop over the batches
        for i in range(No_of_batches):
                
            y_hat[i] = predict_h(full_features[i],theta)
            g = get_gradient(full_features[i],full_target[i] ,theta)
            v_rmsprop = beta_rmsprop*v_rmsprop + (1-beta_rmsprop)*g**2

            theta = theta - learning_rate*g*(1/(np.sqrt(v_rmsprop)+1e-5))
            thetas.append(theta)
    
            # for full batch or small batches cases
            if (batch_size >=2):
                y_hat[i] = predict_h(full_features[i],theta)
                accrs.append(round(skm.r2_score(full_target[i],y_hat[i]),6))
                
        
        # for 1 by 1 value (Stochastic)
        if (batch_size <2):
            y_hat = np.dot(features, theta)
            accrs.append(round(skm.r2_score(target,y_hat),6))
            #calc the cost
            m = target.shape[0]
            y_hat = predict_h(features,theta)
            loss = y_hat - target
            cost = 1/(2*m)*np.sum(loss**2)
            costs.append(cost)
        iter_time += 1
        
        #get final grad after each iteration to add to list 
        g = get_gradient(full_features[i],full_target[i] ,theta)
        grads.append(g)
        
        if stop_criteria_gradient(g,gradlimit):
            break
#     end the for iteration loop
    
    #for extraction purpose 
    thetas = np.array(thetas).reshape(iter_time,features.shape[1])
    
    #end function 
    return theta,thetas,costs, accrs,iter_time,grads

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_rmsprop(X , Y
                     , number_of_iterations=1000
                     , learning_rate = 0.001
                     , batch_size = None
                     , gradlimit = .00001)
accrs[-1]

### gradient descent  with adam

In [ ]:
def gradient_descent_with_adam(features , target
                     , number_of_iterations=1000
                     , learning_rate = 0.0001
                     , batch_size = None
                     , beta_mAdam = 0
                     , beta_vAdam = 1
                     , gradlimit = .0001):
    thetas = []
    costs = []
    accrs = []
    grads = []
    
    features,batch_size,No_of_batches,full_features,full_target=preparethedata(features,target,batch_size)
    
    #intialize the theta
    theta = np.zeros(features.shape[1]).reshape((features.shape[1],1))
    
    # intialize Y hat for spliting 
    y_hat = full_target.copy()

    v_adam = 0
    m_adam = 0
    #stop to max iter
    iter_time = 0
    #loop over the iterations
    for t in range(number_of_iterations):
        
        #loop over the batches
        for i in range(No_of_batches):
                
            y_hat[i] = predict_h(full_features[i],theta)
            g = get_gradient(full_features[i],full_target[i] ,theta)
            v_adam = v_adam /(1  -beta_vAdam**(t+1) )
            m_adam = m_adam / (1- beta_mAdam**(t+1))
            v_adam = beta_vAdam*v_adam + (1-beta_vAdam)*g**2
            m_adam = beta_mAdam*m_adam + (1-beta_mAdam)*g
            theta = theta - learning_rate*m_adam*(1/(np.sqrt(v_adam)+1e-5))
            thetas.append(theta)
    
            # for full batch or small batches cases
            if (batch_size >=2):
                y_hat[i] = predict_h(full_features[i],theta)
                accrs.append(round(skm.r2_score(full_target[i],y_hat[i]),6))
                
        
        # for 1 by 1 value (Stochastic)
        if (batch_size <2):
            y_hat = np.dot(features, theta)
            accrs.append(round(skm.r2_score(target,y_hat),6))
            #calc the cost
            m = target.shape[0]
            y_hat = predict_h(features,theta)
            loss = y_hat - target
            cost = 1/(2*m)*np.sum(loss**2)
            costs.append(cost)
        iter_time += 1
        
        #get final grad after each iteration to add to list 
        g = get_gradient(full_features[i],full_target[i] ,theta)
        grads.append(g)
        
        if stop_criteria_gradient(g,gradlimit):
            break
#     end the for iteration loop
    
    #for extraction purpose 
    thetas = np.array(thetas).reshape(iter_time,features.shape[1])
    
    #end function 
    return theta,thetas,costs, accrs,iter_time,grads

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_adam(X , Y
                     , number_of_iterations=1000
                     , learning_rate = .155
                     , batch_size = None
                     , beta_mAdam = .9
                     , beta_vAdam = .9
                     , gradlimit = .00001)
accrs[-1]

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_adam(X , Y
                     , number_of_iterations=1000
                     , learning_rate = .155
                     , batch_size = None
                     , beta_mAdam = .9
                     , beta_vAdam = .99
                     , gradlimit = .00001)
accrs[-1]

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_adam(X , Y
                     , number_of_iterations=1000
                     , learning_rate = .155
                     , batch_size = None
                     , beta_mAdam = .6
                     , beta_vAdam = .99
                     , gradlimit = .00001)
accrs[-1]

In [ ]:
theta,thetas,costs, accrs,iter_time,grads = gradient_descent_with_adam(X , Y
                     , number_of_iterations=1000
                     , learning_rate = 1
                     , batch_size = None
                     , beta_mAdam = .9
                     , beta_vAdam = .9
                     , gradlimit = .00001)
accrs[-1]